In [ ]:
import pyspark.pandas as ps

In [ ]:
path = 'dbfs:/FileStore/dados_transformados/data.parquet'
df_data = ps.read_parquet(path)

In [ ]:
df_data.info()

In [ ]:
df_data = df_data.dropna()

In [ ]:
df_data.info()

In [ ]:
df_data['artists_song'] = df_data.artists + ' - ' + df_data.name

In [ ]:
df_data.head(5)

In [ ]:
df_data.info()

In [ ]:
X = df_data.columns.to_list()
X.remove('artists')
X.remove('id')
X.remove('name')
X.remove('artists_song')
X.remove('release_date')
X

In [ ]:
df_data = df_data.to_spark()

In [ ]:
from pyspark.ml.feature import VectorAssembler

In [ ]:
dados_encoded_vector = VectorAssembler(inputCols=X, outputCol='features').transform(df_data)

In [ ]:
dados_encoded_vector.select('features').show(truncate=False, n=5)

In [ ]:
from pyspark.ml.feature import StandardScaler

In [ ]:
scaler = StandardScaler(inputCol='features', outputCol='features_scaled')
model_scaler = scaler.fit(dados_encoded_vector)
dados_musicas_scaler = model_scaler.transform(dados_encoded_vector)

In [ ]:
dados_musicas_scaler.select('features_scaled').show(truncate=False, n=5)

In [ ]:
k = len(X)
k

In [ ]:
from pyspark.ml.feature import PCA

In [ ]:
pca = PCA(k=k, inputCol='features_scaled', outputCol='pca_features')
model_pca = pca.fit(dados_musicas_scaler)
dados_musicas_pca = model_pca.transform(dados_musicas_scaler)

In [ ]:
model_pca.explainedVariance

In [ ]:
sum(model_pca.explainedVariance) * 100

In [ ]:
lista_valores = [sum(model_pca.explainedVariance[0:i+1]) for i in range(k)]
lista_valores

In [ ]:
import numpy as np

In [ ]:
k = sum(np.array(lista_valores) <= 0.7)
k

In [ ]:
pca = PCA(k=k, inputCol='features_scaled', outputCol='pca_features')
model_pca = pca.fit(dados_musicas_scaler)
dados_musicas_pca_final = model_pca.transform(dados_musicas_scaler)

In [ ]:
dados_musicas_pca_final.select('pca_features').show(truncate=False, n=5)

In [ ]:
from pyspark.ml import Pipeline

In [ ]:
pca_pipeline = Pipeline(stages=[VectorAssembler(inputCols=X, outputCol='features'),
                                StandardScaler(inputCol='features', outputCol='features_scaled'),
                                PCA(k=6, inputCol='features_scaled', outputCol='pca_features')])

In [ ]:
model_pca_pipeline = pca_pipeline.fit(df_data)

In [ ]:
projection = model_pca_pipeline.transform(df_data)

In [ ]:
projection.select('pca_features').show(truncate=False, n=5)

In [ ]:
from pyspark.ml.clustering import KMeans

In [ ]:
SEED = 1224

In [ ]:
kmeans = KMeans(k=50, featuresCol='pca_features', predictionCol='cluster_pca', seed=SEED)

In [ ]:
modelo_kmeans = kmeans.fit(projection)

In [ ]:
projetion_kmeans = modelo_kmeans.transform(projection) 

In [ ]:
projetion_kmeans.select(['pca_features','cluster_pca']).show()

In [ ]:
from pyspark.ml.functions import vector_to_array

In [ ]:
projetion_kmeans = projetion_kmeans.withColumn('x', vector_to_array('pca_features')[0])\
                                   .withColumn('y', vector_to_array('pca_features')[1])

In [ ]:
projetion_kmeans.select(['x', 'y', 'cluster_pca', 'artists_song']).show()

In [ ]:
import plotly.express as px

In [ ]:
fig = px.scatter(projetion_kmeans.toPandas(), x='x', y='y', color='cluster_pca', hover_data=['artists_song'])
fig.show()

In [ ]:
nome_musica = 'Taylor Swift - Blank Space'

In [ ]:
cluster = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
cluster

In [ ]:
musicas_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                       .select('artists_song', 'id', 'pca_features')
musicas_recomendadas.show()

In [ ]:
componenetes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
                                          .select('pca_features').collect()[0][0]
componenetes_musica                             

In [ ]:
from scipy.spatial.distance import euclidean
from pyspark.sql.types import FloatType
import pyspark.sql.functions as f

In [ ]:
def calcula_distance(value):
    return euclidean(componenetes_musica, value)

udf_calcula_distance = f.udf(calcula_distance, FloatType())

musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))

In [ ]:
recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

recomendadas.show()

In [ ]:
def recomendador(nome_musica):
    cluster = projetion_kmeans.filter(projetion_kmeans.artists_song == nome_musica).select('cluster_pca').collect()[0][0]
    musicas_recomendadas = projetion_kmeans.filter(projetion_kmeans.cluster_pca == cluster)\
                                       .select('artists_song', 'id', 'pca_features')
    componenetes_musica = musicas_recomendadas.filter(musicas_recomendadas.artists_song == nome_musica)\
                                          .select('pca_features').collect()[0][0]
    
    def calcula_distance(value):
        return euclidean(componenetes_musica, value)

    udf_calcula_distance = f.udf(calcula_distance, FloatType())

    musicas_recomendadas_dist = musicas_recomendadas.withColumn('Dist', udf_calcula_distance('pca_features'))
    
    recomendadas = spark.createDataFrame(musicas_recomendadas_dist.sort('Dist').take(10)).select(['artists_song', 'id', 'Dist'])

    return recomendadas

In [ ]:
df_recomedada = recomendador('Taylor Swift - Blank Space')
df_recomedada.show()